In [ ]:
%pip install -r requirements.txt

Basic Anthropic Integration

In [ ]:
import anthropic
import json
import random
import string

anthropic_key = "YOUR_ANTHROPIC_KEY"
payi_api_key = "YOUR_PAYI_API_KEY"

payi_headers = {
    "xProxy-Forward-x-api-key": anthropic_key,
    "xProxy-Forward-anthropic-version": "2023-06-01"
}

payi_base_url = "https://api.pay-i.com"
payi_anthropic_url = payi_base_url + "/api/v1/proxy/anthropic"

client = anthropic.Anthropic(
    base_url = payi_anthropic_url,
    auth_token = payi_api_key,
    default_headers=payi_headers
)

message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ]
)
completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))


Use the Pay-i SDK to send a request with request tags

In [ ]:
from payi.lib.helpers import create_headers

message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a request tag test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Create a budget and make a request with that budget

In [ ]:
from payi import Payi

payi_client = Payi(
    payi_api_key=payi_api_key,
    base_url=payi_base_url
)

#Create a budget
budget_response = payi_client.budgets.create(
    #Use a random name for the budget
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)), 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a budget test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        request_tags=["x", "y"],
        budget_ids=[budget_id]
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

See budget status

In [ ]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.output.base}")

Make an ingest call with pre-computed token values

In [ ]:
response = payi_client.ingest.units(
    category="system.anthropic",
    resource="claude-3-sonnet-20240229",
    input=50,
    output=100,
    budget_ids= [budget_id],
    request_tags = ["a", "b"]
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.cost.input.base}")
print(f"Output Base Cost: {response.cost.output.base}")
print(f"Total Base Cost: {response.cost.total.base}")

Reset a budget back to zero tracked cost

In [ ]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.budget_reset_timestamp}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total.base}")

List and then delete all budgets

In [ ]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)